# Digital Video Stabilization

In [1]:
import numpy as np
import cv2
import os
from time import time
from tqdm import tqdm
import matplotlib.pyplot as plt

### Set Input and Output Videos

In [2]:
# Read input video
input_video = 'Data/piano.mp4'
cap = cv2.VideoCapture(input_video)

In [3]:
# Get frame count
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Get fps
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [4]:
# Define the codec for output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')

In [5]:
SMOOTHING_RADIUS = 100 # TODO: Try different values

# Set up output video
out = cv2.VideoWriter('{}_stabilized_{}.avi'.format(input_video.split('.')[0], SMOOTHING_RADIUS), fourcc, fps, (w, h))

### Read the first frame and convert it to grayscale

In [6]:
# Read first frame
_, prev = cap.read() 
 
# Convert frame to grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

### Find motion between frames

In [7]:
# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32) 

In [8]:
# for i in tqdm(range(n_frames-2)):
for i in tqdm(range(57)):
    # Detect feature points in previous frame
    prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                       maxCorners=200,
                                       qualityLevel=0.01,
                                       minDistance=30,
                                       blockSize=3)
    # Read next frame
    success, curr = cap.read() 
    if not success: 
        print("Finished the whole video")
        break
    
    # Convert to grayscale
    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    # Calculate optical flow (i.e. track feature points)
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 
 
    # Sanity check
    assert prev_pts.shape == curr_pts.shape 
    
    # Filter only valid points (the ones that are not occulated or out of frame)
    idx = np.where(status==1)[0]
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]
    
    #Find transformation matrix
    m = cv2.estimateAffinePartial2D(prev_pts, curr_pts)[0]
    
    # Extract traslation
    dx = m[0,2]
    dy = m[1,2]

    # Extract rotation angle
    da = np.arctan2(m[1,0], m[0,0])

    # Store transformation
    transforms[i] = [dx,dy,da]

    # Move to next frame
    prev_gray = curr_gray    

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 79.16it/s]


### Calculate smooth motion between frames

In [9]:
# Compute trajectory using cumulative sum of transformations
trajectory = np.cumsum(transforms, axis=0)

In [10]:
def movingAverage(curve, radius): 
    window_size = 2 * radius + 1
    # Define the filter 
    f = np.ones(window_size)/window_size 
    # Add padding to the boundaries 
    curve_pad = np.lib.pad(curve, (radius, radius), 'edge') 
    # Apply convolution 
    curve_smoothed = np.convolve(curve_pad, f, mode='same') 
    # Remove padding 
    curve_smoothed = curve_smoothed[radius:-radius]
    # return smoothed curve
    return curve_smoothed

In [11]:
def smooth(trajectory): 
    smoothed_trajectory = np.copy(trajectory) 
    # Filter the x, y and angle curves
    for i in range(3):
        smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)

    return smoothed_trajectory

In [12]:
smoothed_trajectory = smooth(trajectory)

In [13]:
# Calculate difference in smoothed_trajectory and trajectory
difference = smoothed_trajectory - trajectory
  
# Calculate newer transformation array
transforms_smooth = transforms + difference

### Apply smoothed camera motion to frames

In [14]:
def fixBorder(frame):
    s = frame.shape
    # Scale the image 10% without moving the center
    T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.1)
    frame = cv2.warpAffine(frame, T, (s[1], s[0]))
    return frame

In [15]:
# Reset stream to first frame 
cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 
max_up = 0
# Write n_frames-1 transformed frames
for i in tqdm(range(n_frames-2)):
    # Read next frame
    success, frame = cap.read() 
    if not success:
        break

    # Extract transformations from the new transformation array
    dx = transforms_smooth[i,0]
    dy = transforms_smooth[i,1]
    da = transforms_smooth[i,2]

    # Reconstruct transformation matrix accordingly to new values
    m = np.zeros((2,3), np.float32)
    m[0,0] = np.cos(da)
    m[0,1] = -np.sin(da)
    m[1,0] = np.sin(da)
    m[1,1] = np.cos(da)
    m[0,2] = dx
    m[1,2] = dy

    # Apply affine wrapping to the given frame
    frame_stabilized = cv2.warpAffine(frame, m, (w,h))
    
    # Fix border artifacts
    frame_stabilized = fixBorder(frame_stabilized) 
#     # Write the frame to the file
#     frame_out = cv2.hconcat([frame, frame_stabilized])

    out.write(frame_stabilized)
    
out.release()

100%|█████████████████████████████████████████████████████████████████████████████| 1284/1284 [00:04<00:00, 260.98it/s]
